In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import math

In [2]:
review_distribution = pd.read_json('../full_dataset/visit_distribution.json')
review_distribution['merge_token'] = review_distribution.apply(lambda x: x['authority_id'] + ',' + str(x['authority_stars']) + ',' + str(x['distance']), axis = 1)
review_distribution.head()

,0,authority_id,authority_stars,distance,time_distribution,merge_token
0,1,-2RVxn8RRN3j9bOiqGkjug,5,2,0,"-2RVxn8RRN3j9bOiqGkjug,5,2"
1,4,-2RVxn8RRN3j9bOiqGkjug,5,4,0,"-2RVxn8RRN3j9bOiqGkjug,5,4"
10,56,-4AMRN-7RtozD9s8ZsH04g,4,3,0,"-4AMRN-7RtozD9s8ZsH04g,4,3"
100,54,0bNXP9quoJEgyVZu9ipGgQ,4,3,0,"0bNXP9quoJEgyVZu9ipGgQ,4,3"
1000,1,NjOYSgr2LNMoSPi4e140Ig,2,5,0,"NjOYSgr2LNMoSPi4e140Ig,2,5"


In [3]:
d_before = DataFrame(review_distribution.loc[review_distribution['time_distribution'] == 0])
d_before.columns=['count_before', 'authority_id', 'authority_stars', 'distance', 'time_distribution', 'merge_token']
d_after = DataFrame(review_distribution.loc[review_distribution['time_distribution'] == 1][['merge_token', '0']])
d_after.columns = ['merge_token', 'count_after']

In [4]:
factor = pd.merge(d_before, d_after, on = 'merge_token')[['authority_id', 'authority_stars', 'distance', 'count_before', 'count_after']]
factor['factor'] = factor.apply(lambda x: x['count_after'] / x['count_before'], axis = 1)
factor.head()

,authority_id,authority_stars,distance,count_before,count_after,factor
0,-2RVxn8RRN3j9bOiqGkjug,5,4,4,4,1.000000
1,-4AMRN-7RtozD9s8ZsH04g,4,3,56,69,1.232143
2,0bNXP9quoJEgyVZu9ipGgQ,4,3,54,115,2.129630
3,NjOYSgr2LNMoSPi4e140Ig,3,5,4,12,3.000000
4,NjOYSgr2LNMoSPi4e140Ig,4,5,1,7,7.000000


In [5]:
aggregate_result = factor[['authority_stars', 'distance', 'factor']].groupby(['authority_stars', 'distance'], as_index = False)['factor'].agg({'size': np.size, 'factor': np.mean, 'sigma': np.std })
aggregate_result.loc[aggregate_result['size'] > 3]

,authority_stars,distance,factor,size,sigma
0,1,2,1.593538,21,1.702245
1,1,3,2.548335,31,3.148726
2,1,4,7.257135,29,14.064348
3,1,5,5.440852,19,6.421885
5,2,2,2.079316,51,3.023513
6,2,3,5.819026,47,12.466931
7,2,4,5.926495,44,9.946139
8,2,5,4.748419,23,7.978910
9,3,1,0.925926,9,0.486706
10,3,2,1.657321,78,2.464113


In [6]:
factor['weight'] = factor.apply(lambda x: x['count_before'] + x['count_after'], axis = 1)
def weighted_avg_and_std(values):
    """
    Return the weighted average and biased weighted standard deviation.

    Adapted from: http://stackoverflow.com/questions/2413522/weighted-standard-deviation-in-numpy
    """
    
    average = np.average(values, weights=factor.loc[values.index, 'weight'])
    variance = np.average((values-average)**2, weights=factor.loc[values.index, 'weight'])  # Fast and numerically precise
    return (average, math.sqrt(variance))

In [7]:
agg_weighted = factor[['authority_stars', 'distance', 'factor', 'weight']].groupby(['authority_stars', 'distance'], as_index = False).agg({ 'factor': weighted_avg_and_std, 'weight': np.sum })
agg_weighted.loc[aggregate_result['size'] > 3]

,authority_stars,distance,factor,weight
0,1,2,"(1.40226566599, 1.2453013469512981)",369
1,1,3,"(4.13426153518, 3.3188086363410934)",2626
2,1,4,"(11.3617156454, 15.511906231600634)",1595
3,1,5,"(10.7890386343, 9.748160226704673)",212
5,2,2,"(2.20590614719, 3.224140412446293)",984
6,2,3,"(5.05715346266, 9.535559307950633)",5315
7,2,4,"(5.63663830836, 8.483005228793417)",3073
8,2,5,"(8.90021810672, 11.090482218462153)",392
9,3,1,"(0.884057971014, 0.5094156513281718)",23
10,3,2,"(1.83038314403, 2.6970012388521094)",3152
